In [333]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [334]:
#for future use for 1 million rating testing
movies_info= pd.read_csv('/content/drive/My Drive/data_mining project/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users_info =pd.read_csv('/content/drive/My Drive/data_mining project/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings_info =pd.read_csv('/content/drive/My Drive/data_mining project/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [335]:
training_set=pd.read_csv('/content/drive/My Drive/data_mining project/ml-100k/u1.base', delimiter = '\t')
training_set=np.array(training_set, dtype = 'int')
test_set=pd.read_csv('/content/drive/My Drive/data_mining project/ml-100k/u1.test', delimiter = '\t')
test_set=np.array(test_set, dtype = 'int')

In [336]:
user_count =int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
movies_count =int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [337]:
def modify(data):
  listoflist = []
  for ui in range(1, user_count+1):
    mi =data[:, 1][data[:, 0]==ui]
    ri =data[:, 2][data[:, 0]==ui]
    listofratings =np.zeros(movies_count)
    listofratings[mi-1] =ri
    listoflist.append(list(listofratings))
  return listoflist
training_set= modify(training_set)
test_set= modify(test_set)

In [338]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [339]:
class MODELL(nn.Module):
    def __init__(self, ):
        super(MODELL, self).__init__()
        self.l1 =nn.Linear(movies_count, 20)
        self.l2 =nn.Linear(20, 10)
        self.l3 =nn.Linear(10, 20)
        self.l4 =nn.Linear(20, movies_count)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x =self.activation(self.l1(x))
        x =self.activation(self.l2(x))
        x =self.activation(self.l3(x))
        x =self.l4(x)
        return x
archi =MODELL()
criterion =nn.MSELoss()
optimizer =optim.RMSprop(archi.parameters(), lr = 0.01, weight_decay = 0.5)

In [340]:
e_counts=200
for epoch in range(e_counts):
  train_loss , s =0 , 0.
  for ui in range(user_count):
    input_info = Variable(training_set[ui]).unsqueeze(0)
    target_info = input_info.clone()
    if torch.sum(target_info.data > 0) > 0:
      output_info =archi(input_info)
      target_info.require_grad =False
      output_info[target_info ==0] =0
      l =criterion(output_info, target_info)
      adj_cons =movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
      l.backward()
      train_loss +=np.sqrt(l.data*adj_cons)
      s +=1.
      optimizer.step()
  print('loss at epoch number '+ str(epoch+1)+ ' is ' +str(train_loss/s))

loss at epoch number 1 is tensor(1.7714)
loss at epoch number 2 is tensor(1.0967)


KeyboardInterrupt: ignored

In [ ]:
test_loss ,s  = 0 , 0.
 
for ui in range(user_count):
  input_info= Variable(training_set[ui]).unsqueeze(0)
  target_info =Variable(test_set[ui]).unsqueeze(0)
  if torch.sum(target_info.data>0)> 0:
    output_info =archi(input_info)
    target_info.require_grad= False
    output_info[target_info== 0] =0
    l= criterion(output_info, target_info)
    adj_cons =movies_count/float(torch.sum(target_info.data > 0) + 1e-10)
    test_loss += np.sqrt(l.data*adj_cons)
    s+= 1.
print('test loss: '+str(test_loss/s))